# WISER SIMS Monthly Course Evaluation Comparison

This takes a comma separated data file (generated from the SIMS database using the query below), and creates bar charts comparing a courses standard evaluation questions. Any courses that have responses to the standard evaluation questions for classes in the date range will get a pdf file with the graph, one, one graph per file.
- Author  : John Lutz <lutzjw@upmc.edu>
- Created : 2020-05-26
- Edited : 2020-05-26

## Some Assumptions
- You have an `Analysis/Course-Evaluations` directory in the folder your are running this Jupyter notebook. This is where the PDF files containing the graphs will go.
- You have a datafile generated from the SIMS database using the query below in "__Query to pull data__"

## Instructions

- Change the variables in the section below. The ones you will need to always change are:
    - `file`
        - This is the comma separated data file that is generated from the query above. It is individual answers to the standard course evaluation questions, one answer per line. Typically we can just collect the data for the previous month and append it to this data file.
    - `startDate` and `endDate`
        - This is the start and end dates for the classes you want to generate the charts for. Somebody may complete an evaluation well after the class has occured, so there may be some anomylous data over time.
    - `directoryName`
        - This is the name of the directory that will be created in `Analysis/Course-Evaluations`. This is where all of the PDFs of the graphs wil go. The directory will be created if needed. Any existing files will be overwritten if the same file names are generated.
    - `thisCourseOnly` (optional)
        - This specifies that you should create graphs for ONLY this course. The name must match __EXACTLY__. Set this variable to double quotes if you want all courses in the time range.

- Once you have made your changes hit the **SHIFT-RETURN** keys together to run the analysis
- Scroll to the bottom to see the results
- the Mac has a command line utility to combine PDFS into one PDF file:

`"/System/Library/Automator/Combine PDF Pages.action/Contents/Resources/join.py" -o outfile.pdf pdf1.pdf pdf2.pdf pdf3.pdf...`

# Query to pull data    

- SQL to run on SIMS data:
      
```
select c.ABBRV COURSE, c.COURSE_ID, to_char(l.CLASS_DATE, 'YYYY-MM-DD') CLASS_DATE,
       l.CLASS_ID, a.EVAL_ANSWER_ID SCORE, to_char(m.EVALUATE_DATE, 'YYYY-MM-DD HH24:MI:SS') EVALUATE_DATE,
       case when instr(q.QUESTION_TEXT, 'Effectiveness') > 1 then 'E'
            when instr(q.QUESTION_TEXT, 'Likeliness')    > 1 then 'L'
       else 'F' END "TYPE" --Facilitator
  from EVALUATION_ANSWERS a, EVALUATION_MAIN m, CLASSES l, courses c, ID0_EVAL_QUESTIONS q
 where a.EVALUATION_ID = m.EVALUATION_ID
   and a.EVAL_QUESTION_ID = q.EVAL_QUESTION_ID
   and m.CLASS_ID = l.CLASS_ID
   and l.COURSE_ID = c.COURSE_ID
   -- This will pull the data from last month (if you run it before the 28th of the month).
   and trunc(m.EVALUATE_DATE) between trunc(sysdate-28, 'mm')
                                  and trunc(sysdate,    'mm')-1
--   and trunc(m.EVALUATE_DATE) < to_date('2020-05-01', 'YYYY-MM-DD')
   -- Effectiveness of the in-person education you received
   and (a.EVAL_QUESTION_ID in (36277, 36336, 36348, 36613, 36858, 36927, 37139, 37180, 37217, 37240, 37284, 37406, 37418, 37724, 37966, 38069, 38108, 38220)
       -- Likeliness of recommending this course to a colleague
        or a.EVAL_QUESTION_ID in (36855, 36924, 37136, 37176, 37192, 37194, 37214, 37237, 37281, 37299, 37403, 37415, 37721, 37831, 37951, 37963, 38066, 38105, 38219)
      -- The facilitator(s) made the educational experience relevant to my training level
        or a.EVAL_QUESTION_ID in (36280, 36339, 36351, 36616, 36861, 36930, 37142, 37183, 37220, 37244, 37287, 37409, 37421, 37727, 37969, 38072, 38111, 38223)
        )
order by EVALUATE_DATE
```

In [ ]:
%matplotlib inline
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import datetime
import cx_Oracle

myJupyterLocation = '/Users/johnlutz/Library/Mobile Documents/com~apple~CloudDocs/Code/Jupyter Notebooks/'
numberOfWeeksBack = 24

#Comma Separated Value file from SIMS using the Query Above.
file = "WISER/data/All Course Analysis/ALL Course Data since 2019-07.csv"

#Beginning and ending class dates, inclusive, of the data you want to graph
today = datetime.datetime.today()
#Get the first day of this month
thisMonth = today.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
#find last month start and end dates
#!!THIS WILL ONLY WORK IF YOU RUN THIS BEFORE THE 28th OF THE MONTH!!!
lastMonth      = today - datetime.timedelta(weeks=numberOfWeeksBack)
lastMonthStart = lastMonth.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
lastMonthEnd   = thisMonth - datetime.timedelta(days=1)

#Name of the directory to, potentially create, and store the graphs in.
#This directory will be in ./Analysis/Course-Evaluations
directoryName = lastMonth.strftime('%Y-%m')
print (directoryName)

# These are the questions that we'll be graphing.
# The dictionary key does *not* need to be one character.
questionDict = {'E' : 'Effectiveness of the in-person education you received',
                'L' : 'Likeliness of recommending this course to a colleague', 
                'F' : 'The facilitator(s) made the educational experience relevant to my training level'
               }

#Make the output directory for the PDF files
outFilePath = "Analysis/Course-Evaluations/" +directoryName+"/"
Path(outFilePath).mkdir(parents=True, exist_ok=True)
print ("Creating PDF files in : " +outFilePath)

#Get last month's data from the Oracle Database
dsn_tns    = cx_Oracle.makedsn("y27prd01.isd.upmc.edu",1521, "SIMP2")
connection = cx_Oracle.connect('simmedical_prod', 'prod7892', "y27prd01.isd.upmc.edu:1521/SIMP2")
selectStr = "SELECT * FROM DUAL"
lastDF = pd.read_sql_query(selectStr, con=credentials)
lastDF.head()

#Add my home directory to the path and get the full file name
file = myJupyterLocation + file
base = os.path.basename(file)
fileName=os.path.splitext(base)[0]

#Load the file into a Dataframe
allData = pd.read_csv(file)
#makes dates be dates
allData['CLASS_DATE'] = pd.to_datetime(allData['CLASS_DATE'])
allData['EVALUATE_DATE'] = pd.to_datetime(allData['EVALUATE_DATE'])
#clean up the data
allData.replace(-999.0, np.NaN, inplace=True)
allData.dropna(inplace=True)
allData.drop_duplicates()

print("Reading {} records. ".format(allData.shape[0]))

#Pull just last months data out of this.
lastMonthDF = allData[(allData['CLASS_DATE']>=lastMonthStart) & (allData['CLASS_DATE']<=lastMonthEnd)].copy()
print("{0:} records beween {1:} and {2:}.".format(lastMonthDF.shape[0], lastMonthStart, lastMonthEnd))

#These are the headers of the columns for the ce dataframe that we will generate the plots from
theColumnList = ["the1s","the2s","the3s","the4s","the5s"]

# Pivot, compressing the scores to counts of each score (1-5)
pivoted = lastMonthDF.pivot_table(index=["COURSE", "TYPE"],
                                  columns="SCORE",
                                  aggfunc={'SCORE':np.count_nonzero}
                                 )
# will be the compressed, pivoted data
lastMonthsTotals = pd.DataFrame(pivoted.to_records())
lastMonthsTotals.fillna(0, inplace=True) #clean out the NaNs

#clean out the header cruft from when we created the Pivot Table.
lastMonthsTotals.columns = [hdr.replace("('SCORE', ", "").replace(")", "") for hdr in lastMonthsTotals.columns]

#Need to rename the columns to alphnumeric names to reference them below
lastMonthsTotals.rename(columns={"1.0":"the1s", "2.0":"the2s", "3.0":"the3s", "4.0":"the4s", "5.0":"the5s"}, inplace=True)

#There may be no data for some of the columns (no 1s for example),
#so we may need to create columns with zeros avoid errors below.
colCnt = 2
for col in theColumnList : 
    if col not in lastMonthsTotals.columns :
        lastMonthsTotals.insert(colCnt, col, 0)
    colCnt += 1

#Create a new TOTAL column which is a sum of the columns 1-5
lastMonthsTotals.eval('TOTAL=@lastMonthsTotals.the1s+@lastMonthsTotals.the2s+@lastMonthsTotals.the3s+@lastMonthsTotals.the4s+@lastMonthsTotals.the5s', inplace=True)    

# print (lastMonthsTotals)

#these will be the columns for the new totalsDF Dataframe
totalsColumns = theColumnList+ ["TOTAL"]
lastMonthSumTotalsDF = pd.DataFrame(columns=totalsColumns, index=questionDict.keys())
#Calculate the totals of each column and put it in a dataframe.
#We will use this to get the percentage for each course.
for key in questionDict : 
    totalAll = 0
    for hdr in theColumnList :
        #Calculate the totals
        lastMonthSumTotalsDF.loc[key,hdr] = lastMonthsTotals[(lastMonthsTotals.TYPE==key)][hdr].sum()
        totalAll +=  lastMonthSumTotalsDF.loc[key,hdr]
    lastMonthSumTotalsDF.loc[key, "TOTAL"] = totalAll

# print("The lastMonthSumTotalsDF is : ")
# print(lastMonthSumTotalsDF)

#Get the list of courses.
courseList = lastMonthDF.COURSE.unique()
# print(courseList)
#this is the X Axis for the graphs
x = np.array([1, 2, 3, 4, 5])

#Set some counts.
plotCnt   =  0
courseCnt = 0
for course in courseList :
    plt.style.use('seaborn-deep') #Nice dark style - seaborn-deep
    fig, axs = plt.subplots(3, sharex=True)
    fig.suptitle(course+"\n"+lastMonthStart.strftime('%y-%m-%d')+ " - " +lastMonthEnd.strftime('%y-%m-%d')+ "\n")
    fig.set_size_inches(7.5,10)
#         fig.xlabel('Likert 1-5')
    axCnt = 0
    barWidth = .40
    for key in questionDict :
#         print('Key is : ' +key)
        # Reset the percentage arrays
        #filter to get a dataframe with just this course and this question
        thisCourse      = lastMonthDF[(lastMonthDF.COURSE==course) & (lastMonthDF.TYPE==key)]
        allOtherCourses = lastMonthDF[(lastMonthDF.COURSE!=course) & (lastMonthDF.TYPE==key)]
#         print("This Course:")
#         print(thisCourse)
#         print("All Other Courses:")
#         print(allOtherCourses)
        
#         print ('-----------------------------')
